Day 75: Android App Store (Data Visualization with Plotly)

# Introduction

In this notebook, we will do a comprehensive analysis of the Android app market by comparing thousands of apps in the Google Play store.

# About the Dataset of Google Play Store Apps & Reviews

**Data Source:** <br>
App and review data was scraped from the Google Play Store by Lavanya Gupta in 2018. Original files listed [here](
https://www.kaggle.com/lava18/google-play-store-apps).

# Import Statements

In [232]:
import pandas as pd
import plotly.express as px

# Notebook Presentation

In [233]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format

# Read the Dataset

In [234]:
df_apps = pd.read_csv('apps.csv')

# Data Cleaning

**Challenge**: How many rows and columns does `df_apps` have? What are the column names? Look at a random sample of 5 different rows with [.sample()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html).

In [235]:
df_apps.shape

(10841, 12)

In [236]:
df_apps.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size_MBs', 'Installs', 'Type',
       'Price', 'Content_Rating', 'Genres', 'Last_Updated', 'Android_Ver'],
      dtype='object')

In [237]:
df_apps.sample(5)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Last_Updated,Android_Ver
1792,WISE- MOBILE PORTAL,TOOLS,NaN,3,1.00,500,Free,0,Everyone,Tools,"July 30, 2015",2.2 and up
4252,Sniper Traning for CS GO,GAME,3.20,655,14.00,"50,000",Free,0,Teen,Action,"May 18, 2016",4.0 and up
9575,Phone Tracker : Family Locator,SOCIAL,4.30,231446,5.40,"10,000,000",Free,0,Everyone,Social,"August 2, 2018",4.0 and up
2125,BP Journal - Blood Pressure Diary,MEDICAL,5.00,6,26.00,"1,000",Free,0,Everyone,Medical,"May 25, 2018",4.4 and up
2147,dt Pro,FINANCE,4.80,4,4.50,"1,000",Free,0,Everyone,Finance,"May 16, 2017",2.2 and up


### Drop Unused Columns

**Challenge**: Remove the columns called `Last_Updated` and `Android_Version` from the DataFrame. We will not use these columns. 

In [238]:
df_apps_short = df_apps.drop(columns=['Last_Updated', 'Android_Ver'])

### Find and Remove NaN values in Ratings

**Challenge**: How may rows have a NaN value (not-a-number) in the Ratings column? Create DataFrame called `df_apps_clean` that does not include these rows. 

In [239]:
df_apps_clean = df_apps_short.dropna(subset=['Rating'])

In [240]:
df_apps_clean.shape

(9367, 10)

### Find and Remove Duplicates

**Challenge**: Are there any duplicates in data? Check for duplicates using the [.duplicated()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html) function. How many entries can you find for the "Instagram" app? Use [.drop_duplicates()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html) to remove any duplicates from `df_apps_clean`. 


In [241]:
df_apps_clean[df_apps_clean['App']=='Instagram']

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10808,Instagram,SOCIAL,4.50,66577446,5.30,"1,000,000,000",Free,0,Teen,Social
10809,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10810,Instagram,SOCIAL,4.50,66509917,5.30,"1,000,000,000",Free,0,Teen,Social


In [242]:
df_apps_clean.duplicated().sum()

476

In [243]:
df_apps_clean.duplicated(subset=['App', 'Type', 'Price']).sum()

1168

In [244]:
df_apps_dedupe = df_apps_clean.drop_duplicates(subset=['App', 'Type', 'Price'])

In [245]:
df_apps_dedupe.shape

(8199, 10)

# Find Highest Rated Apps

**Challenge**: Identify which apps are the highest rated. What problem might you encounter if you rely exclusively on ratings alone to determine the quality of an app?

In [246]:
df_apps_dedupe.sort_values(['Rating', 'Reviews'], ascending=False)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
2095,Ríos de Fe,LIFESTYLE,5.00,141,15.00,"1,000",Free,0,Everyone,Lifestyle
2438,"FD Calculator (EMI, SIP, RD & Loan Eligilibility)",FINANCE,5.00,104,2.30,"1,000",Free,0,Everyone,Finance
3115,Oración CX,LIFESTYLE,5.00,103,3.80,"5,000",Free,0,Everyone,Lifestyle
2107,Barisal University App-BU Face,FAMILY,5.00,100,10.00,"1,000",Free,0,Everyone,Education
2069,Master E.K,FAMILY,5.00,90,19.00,"1,000",Free,0,Everyone,Education
...,...,...,...,...,...,...,...,...,...,...
728,Truck Driving Test Class 3 BC,FAMILY,1.00,1,2.00,50,Paid,$1.49,Everyone,Education
1208,Thistletown CI,PRODUCTIVITY,1.00,1,6.60,100,Free,0,Everyone,Productivity
1271,MbH BM,MEDICAL,1.00,1,2.30,100,Free,0,Everyone,Medical
1314,CR Magazine,BUSINESS,1.00,1,7.80,100,Free,0,Everyone,Business


In [247]:
print('Ratings alone are not enough to measure app quality because an app with one download and a 5-star review would not be considered high quality.')

Ratings alone are not enough to measure app quality because an app with one download and a 5-star review would not be considered high quality.


# Find 5 Largest Apps in terms of Size (MBs)

**Challenge**: What's the size in megabytes (MB) of the largest Android apps in the Google Play Store. Based on the data, do you think there could be limit in place or can developers make apps as large as they please? 

In [248]:
df_apps_dedupe.sort_values('Size_MBs', ascending=False)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
9942,Talking Babsy Baby: Baby Games,LIFESTYLE,4.00,140995,100.00,"10,000,000",Free,0,Everyone,Lifestyle;Pretend Play
10687,Hungry Shark Evolution,GAME,4.50,6074334,100.00,"100,000,000",Free,0,Teen,Arcade
9943,Miami crime simulator,GAME,4.00,254518,100.00,"10,000,000",Free,0,Mature 17+,Action
9944,Gangster Town: Vice District,FAMILY,4.30,65146,100.00,"10,000,000",Free,0,Mature 17+,Simulation
3144,Vi Trainer,HEALTH_AND_FITNESS,3.60,124,100.00,"5,000",Free,0,Everyone,Health & Fitness
...,...,...,...,...,...,...,...,...,...,...
2648,Ad Remove Plugin for App2SD,PRODUCTIVITY,4.10,66,0.02,"1,000",Paid,$1.29,Everyone,Productivity
5798,ExDialer PRO Key,COMMUNICATION,4.50,5474,0.02,"100,000",Paid,$3.99,Everyone,Communication
2684,My baby firework (Remove ad),FAMILY,4.10,30,0.01,"1,000",Paid,$0.99,Everyone,Entertainment
7966,Market Update Helper,LIBRARIES_AND_DEMO,4.10,20145,0.01,"1,000,000",Free,0,Everyone,Libraries & Demo


In [249]:
print('It looks like there is a 100MB cap on the size an app can be.')

It looks like there is a 100MB cap on the size an app can be.


# Find the 5 App with Most Reviews

**Challenge**: Which apps have the highest number of reviews? Are there any paid apps among the top 50?

In [250]:
df_apps_dedupe.sort_values('Reviews', ascending=False)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10805,Facebook,SOCIAL,4.10,78158306,5.30,"1,000,000,000",Free,0,Teen,Social
10785,WhatsApp Messenger,COMMUNICATION,4.40,69119316,3.50,"1,000,000,000",Free,0,Everyone,Communication
10806,Instagram,SOCIAL,4.50,66577313,5.30,"1,000,000,000",Free,0,Teen,Social
10784,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56642847,3.50,"1,000,000,000",Free,0,Everyone,Communication
10650,Clash of Clans,GAME,4.60,44891723,98.00,"100,000,000",Free,0,Everyone 10+,Strategy
...,...,...,...,...,...,...,...,...,...,...
453,Wowkwis aq Ka'qaquj,FAMILY,5.00,1,49.00,10,Free,0,Everyone,Education;Education
462,CB Fit,HEALTH_AND_FITNESS,5.00,1,7.80,10,Free,0,Everyone,Health & Fitness
901,ES Billing System (Offline App),PRODUCTIVITY,5.00,1,4.20,100,Free,0,Everyone,Productivity
1416,Ek Kahani Aisi Bhi Season 3 - The Horror Story,FAMILY,3.00,1,5.80,100,Free,0,Teen,Entertainment


In [251]:
df_apps_top = df_apps_dedupe.sort_values('Reviews', ascending=False)
df_apps_top.head(50).sort_values('Price', ascending=False)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10805,Facebook,SOCIAL,4.10,78158306,5.30,"1,000,000,000",Free,0,Teen,Social
10649,Asphalt 8: Airborne,GAME,4.50,8389714,92.00,"100,000,000",Free,0,Teen,Racing
10388,Minion Rush: Despicable Me Official Game,GAME,4.50,10216538,36.00,"100,000,000",Free,0,Everyone 10+,Casual;Action & Adventure
10694,Yes day,GAME,4.50,10055521,94.00,"100,000,000",Free,0,Everyone,Casual
10695,Hay Day,FAMILY,4.50,10053186,94.00,"100,000,000",Free,0,Everyone,Casual
10644,Dream League Soccer 2018,GAME,4.60,9882639,74.00,"100,000,000",Free,0,Everyone,Sports
10696,My Talking Angela,GAME,4.50,9881829,99.00,"100,000,000",Free,0,Everyone,Casual
10660,VivaVideo - Video Editor & Photo Movie,VIDEO_PLAYERS,4.60,9879473,40.00,"100,000,000",Free,0,Teen,Video Players & Editors
10786,Google Chrome: Fast & Secure,COMMUNICATION,4.30,9642995,3.50,"1,000,000,000",Free,0,Everyone,Communication
10817,Maps - Navigate & Explore,TRAVEL_AND_LOCAL,4.30,9235155,12.00,"1,000,000,000",Free,0,Everyone,Travel & Local


In [252]:
print('There are no paid apps in the top 50')

There are no paid apps in the top 50


# Plotly Pie and Donut Charts - Visualise Categorical Data: Content Ratings

In [253]:
ratings = df_apps_dedupe['Content_Rating'].value_counts()
ratings

Everyone           6621
Teen                912
Mature 17+          357
Everyone 10+        305
Adults only 18+       3
Unrated               1
Name: Content_Rating, dtype: int64

In [254]:
fig = px.pie(labels=ratings.index, values=ratings.values, title='Content Rating', names=ratings.index, hole=0.6)
fig.update_traces(textposition='outside', textfont_size=15, textinfo='percent+label')
fig.show()

# Numeric Type Conversion: Examine the Number of Installs

**Challenge**: How many apps had over 1 billion (that's right - BILLION) installations? How many apps just had a single install? 

Check the datatype of the Installs column.

Count the number of apps at each level of installations. 

Convert the number of installations (the Installs column) to a numeric data type. Hint: this is a 2-step process. You'll have make sure you remove non-numeric characters first. 

In [255]:
type(df_apps_dedupe['Installs'].iloc[0])

str

In [256]:
installs = [item.replace(',', '') for item in df_apps_dedupe['Installs']]
installs = pd.to_numeric(installs)
df_apps_dedupe['Installs'] = installs
type(df_apps_dedupe['Installs'].iloc[0])

/Users/mymac/PycharmProjects/100DaysOfCode/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



numpy.int64

In [257]:
df_apps_dedupe.sort_values('Installs')

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
21,KBA-EZ Health Guide,MEDICAL,5.00,4,25.00,1,Free,0,Everyone,Medical
28,Ra Ga Ba,GAME,5.00,2,20.00,1,Paid,$1.49,Everyone,Arcade
47,Mu.F.O.,GAME,5.00,2,16.00,1,Paid,$0.99,Everyone,Arcade
162,CB Heroes,SOCIAL,5.00,5,1.80,5,Free,0,Everyone,Social
154,CZ-Help,BOOKS_AND_REFERENCE,5.00,2,1.40,5,Free,0,Everyone,Books & Reference
...,...,...,...,...,...,...,...,...,...,...
10784,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56642847,3.50,1000000000,Free,0,Everyone,Communication
10783,Google Play Books,BOOKS_AND_REFERENCE,3.90,1433233,5.70,1000000000,Free,0,Teen,Books & Reference
10831,Google News,NEWS_AND_MAGAZINES,3.90,877635,13.00,1000000000,Free,0,Teen,News & Magazines
10805,Facebook,SOCIAL,4.10,78158306,5.30,1000000000,Free,0,Teen,Social


In [258]:
df_apps_dedupe.sort_values('Installs', ascending=False)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres
10835,Subway Surfers,GAME,4.50,27722264,76.00,1000000000,Free,0,Everyone 10+,Arcade
10806,Instagram,SOCIAL,4.50,66577313,5.30,1000000000,Free,0,Teen,Social
10783,Google Play Books,BOOKS_AND_REFERENCE,3.90,1433233,5.70,1000000000,Free,0,Teen,Books & Reference
10784,Messenger – Text and Video Chat for Free,COMMUNICATION,4.00,56642847,3.50,1000000000,Free,0,Everyone,Communication
10785,WhatsApp Messenger,COMMUNICATION,4.40,69119316,3.50,1000000000,Free,0,Everyone,Communication
...,...,...,...,...,...,...,...,...,...,...
99,Anatomy & Physiology Vocabulary Exam Review App,MEDICAL,5.00,1,4.60,5,Free,0,Everyone,Medical
82,Brick Breaker BR,GAME,5.00,7,19.00,5,Free,0,Everyone,Arcade
47,Mu.F.O.,GAME,5.00,2,16.00,1,Paid,$0.99,Everyone,Arcade
28,Ra Ga Ba,GAME,5.00,2,20.00,1,Paid,$1.49,Everyone,Arcade


# Find the Most Expensive Apps, Filter out the Junk, and Calculate a (ballpark) Sales Revenue Estimate

Let's examine the Price column more closely.

**Challenge**: Convert the price column to numeric data. Then investigate the top 20 most expensive apps in the dataset.

Remove all apps that cost more than $250 from the `df_apps_clean` DataFrame.

Add a column called 'Revenue_Estimate' to the DataFrame. This column should hold the price of the app times the number of installs. What are the top 10 highest grossing paid apps according to this estimate? Out of the top 10 highest grossing paid apps, how many are games?


In [259]:
price = [p.replace('$','') for p in df_apps_dedupe['Price']]
price = pd.to_numeric(price)
df_apps_dedupe['Price'] = price

/Users/mymac/PycharmProjects/100DaysOfCode/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### The most expensive apps sub $250

In [260]:
df_sub250 = df_apps_dedupe[df_apps_dedupe['Price']<250]
df_sub250['Revenue_Estimate'] = df_sub250['Price']*df_sub250['Installs']
df_sub250.sort_values('Price', ascending=False)

/Users/mymac/PycharmProjects/100DaysOfCode/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Revenue_Estimate
2281,Vargo Anesthesia Mega App,MEDICAL,4.60,92,32.00,1000,Paid,79.99,Everyone,Medical,"79,990.00"
1407,LTC AS Legal,MEDICAL,4.00,6,1.30,100,Paid,39.99,Everyone,Medical,"3,999.00"
2629,I am Rich Person,LIFESTYLE,4.20,134,1.80,1000,Paid,37.99,Everyone,Lifestyle,"37,990.00"
2481,A Manual of Acupuncture,MEDICAL,3.50,214,68.00,1000,Paid,33.99,Everyone,Medical,"33,990.00"
2463,PTA Content Master,MEDICAL,4.20,64,41.00,1000,Paid,29.99,Everyone,Medical,"29,990.00"
...,...,...,...,...,...,...,...,...,...,...,...
4508,myAir™ for Air10™ by ResMed,MEDICAL,3.70,236,18.00,50000,Free,0.00,Everyone,Medical,0.00
4507,AK Math Coach,FAMILY,3.60,283,18.00,50000,Free,0.00,Everyone,Education,0.00
4506,Forgotten Hill: Fall,GAME,4.40,1063,18.00,50000,Free,0.00,Teen,Adventure,0.00
4505,AE Video Poker,GAME,4.00,721,18.00,50000,Free,0.00,Teen,Casino,0.00


### Highest Grossing Paid Apps (ballpark estimate)

In [261]:
df_sub250.sort_values('Revenue_Estimate', ascending=False).head(10)

,App,Category,Rating,Reviews,Size_MBs,Installs,Type,Price,Content_Rating,Genres,Revenue_Estimate
9220,Minecraft,FAMILY,4.50,2376564,19.00,10000000,Paid,6.99,Everyone 10+,Arcade;Action & Adventure,"69,900,000.00"
8825,Hitman Sniper,GAME,4.60,408292,29.00,10000000,Paid,0.99,Mature 17+,Action,"9,900,000.00"
7151,Grand Theft Auto: San Andreas,GAME,4.40,348962,26.00,1000000,Paid,6.99,Mature 17+,Action,"6,990,000.00"
7477,Facetune - For Free,PHOTOGRAPHY,4.40,49553,48.00,1000000,Paid,5.99,Everyone,Photography,"5,990,000.00"
7977,Sleep as Android Unlock,LIFESTYLE,4.50,23966,0.85,1000000,Paid,5.99,Everyone,Lifestyle,"5,990,000.00"
6594,DraStic DS Emulator,GAME,4.60,87766,12.00,1000000,Paid,4.99,Everyone,Action,"4,990,000.00"
6082,Weather Live,WEATHER,4.50,76593,4.75,500000,Paid,5.99,Everyone,Weather,"2,995,000.00"
7954,Bloons TD 5,FAMILY,4.60,190086,94.00,1000000,Paid,2.99,Everyone,Strategy,"2,990,000.00"
7633,Five Nights at Freddy's,GAME,4.60,100805,50.00,1000000,Paid,2.99,Teen,Action,"2,990,000.00"
6746,Card Wars - Adventure Time,FAMILY,4.30,129603,23.00,1000000,Paid,2.99,Everyone 10+,Card;Action & Adventure,"2,990,000.00"


In [262]:
print('7/10 of the top grossing apps are games (some though are classified as "FAMILY" but are in fact games).')

7/10 of the top grossing apps are games (some though are classified as "FAMILY" but are in fact games).


# Plotly Bar Charts & Scatter Plots: Analysing App Categories

In [263]:
df_sub250['Category'].nunique()

33

In [264]:
df_sub250['Category'].value_counts()

FAMILY                 1606
GAME                    910
TOOLS                   719
PRODUCTIVITY            301
PERSONALIZATION         298
LIFESTYLE               297
FINANCE                 296
MEDICAL                 292
PHOTOGRAPHY             263
BUSINESS                262
SPORTS                  260
COMMUNICATION           257
HEALTH_AND_FITNESS      243
NEWS_AND_MAGAZINES      204
SOCIAL                  203
TRAVEL_AND_LOCAL        187
SHOPPING                180
BOOKS_AND_REFERENCE     169
VIDEO_PLAYERS           148
DATING                  134
EDUCATION               118
MAPS_AND_NAVIGATION     118
ENTERTAINMENT           102
FOOD_AND_DRINK           94
AUTO_AND_VEHICLES        73
WEATHER                  72
LIBRARIES_AND_DEMO       64
HOUSE_AND_HOME           62
ART_AND_DESIGN           61
COMICS                   54
PARENTING                50
EVENTS                   45
BEAUTY                   42
Name: Category, dtype: int64

In [265]:
top10_category = df_sub250['Category'].value_counts()[:10]

### Vertical Bar Chart - Highest Competition (Number of Apps)

In [266]:
bar = px.bar(x=top10_category.index, y=top10_category.values)
bar.show()

### Horizontal Bar Chart - Most Popular Categories (Highest Downloads)

In [267]:
category_installs = df_sub250.groupby('Category').agg({'Installs':pd.Series.sum})
category_installs.sort_values('Installs', ascending=True, inplace=True)

In [268]:
h_bar = px.bar(x=category_installs['Installs'], y=category_installs.index, orientation='h', title='Category Popularity')
h_bar.update_layout(xaxis_title='Number of Downloads', yaxis_title='Category')
h_bar.show()

### Category Concentration - Downloads vs. Competition

**Challenge**: 
* First, create a DataFrame that has the number of apps in one column and the number of installs in another:

<img src=https://imgur.com/uQRSlXi.png width="350">

* Then use the [plotly express examples from the documentation](https://plotly.com/python/line-and-scatter/) alongside the [.scatter() API reference](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html)to create scatter plot that looks like this. 

<img src=https://imgur.com/cHsqh6a.png>

*Hint*: Use the size, hover_name and color parameters in .scatter(). To scale the yaxis, call .update_layout() and specify that the yaxis should be on a log-scale like so: yaxis=dict(type='log')

In [269]:
# Create DataFrames
series_cat = df_sub250['Category'].value_counts()
df_cat = pd.DataFrame(series_cat)
df_installs = df_sub250[['Category', 'Installs']].groupby('Category').sum()

# Merge
df_app_installs = df_cat.merge(df_installs, left_on=df_cat.index, right_on=df_installs.index)

# Format
df_app_installs = df_app_installs.rename(columns={'key_0': 'index', 'Category':'App'})
df_app_installs.set_index('index')

,App,Installs
index,,
FAMILY,1606,4437554490
GAME,910,13858762717
TOOLS,719,8099724500
PRODUCTIVITY,301,5788070180
PERSONALIZATION,298,1532352930
LIFESTYLE,297,503611120
FINANCE,296,455249400
MEDICAL,292,39162676
PHOTOGRAPHY,263,4649143130


In [270]:
scatter = px.scatter(df_app_installs, x='App', y='Installs', size='App', color='Installs', title='Category Concentration', hover_name='index')
scatter.update_layout(xaxis_title='Number of Apps (Lower=More Concentrated)', yaxis_title='Installs', yaxis={'type':'log'})
scatter.show()

# Extracting Nested Data from a Column

**Challenge**: How many types of genres are there? Can an app belong to more than one genre? Check what happens when you use .value_counts() on a column with nested values? See if you can work around this problem by using the .split() function and the DataFrame's [.stack() method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.stack.html).


In [271]:
stack = df_sub250['Genres'].str.split(';', expand=True).stack()
genres = stack.value_counts()
genres

Tools                      719
Education                  587
Entertainment              498
Action                     304
Productivity               301
Personalization            298
Lifestyle                  298
Finance                    296
Medical                    292
Sports                     270
Photography                263
Business                   262
Communication              258
Health & Fitness           245
Casual                     216
News & Magazines           204
Social                     203
Simulation                 200
Travel & Local             187
Arcade                     185
Shopping                   180
Books & Reference          171
Video Players & Editors    150
Dating                     134
Puzzle                     124
Maps & Navigation          118
Role Playing               111
Racing                     103
Action & Adventure          96
Strategy                    95
Food & Drink                94
Educational                 93
Adventur

# Colour Scales in Plotly Charts - Competition in Genres

**Challenge**: Can you create this chart with the Series containing the genre data? 

<img src=https://imgur.com/DbcoQli.png width=400>

Try experimenting with the built in colour scales in Plotly. You can find a full list [here](https://plotly.com/python/builtin-colorscales/). 

* Find a way to set the colour scale using the color_continuous_scale parameter. 
* Find a way to make the color axis disappear by using coloraxis_showscale. 

In [272]:
bar = px.bar(genres, title='Top Genres', hover_name=genres.index, color=genres.values, color_continuous_scale='Agsunset')
bar.update_layout(xaxis_title='Genre', yaxis_title='Number of Apps', coloraxis_showscale=False)
bar.show()

# Grouped Bar Charts: Free vs. Paid Apps per Category

In [273]:
df_sub250['Type'].value_counts()

Free    7595
Paid     589
Name: Type, dtype: int64

In [274]:
df_free_paid = df_sub250.groupby(['Category', 'Type'], as_index=False).agg({'App': pd.Series.count})
df_free_paid

,Category,Type,App
0,ART_AND_DESIGN,Free,58
1,ART_AND_DESIGN,Paid,3
2,AUTO_AND_VEHICLES,Free,72
3,AUTO_AND_VEHICLES,Paid,1
4,BEAUTY,Free,42
...,...,...,...
56,TRAVEL_AND_LOCAL,Paid,8
57,VIDEO_PLAYERS,Free,144
58,VIDEO_PLAYERS,Paid,4
59,WEATHER,Free,65


**Challenge**: Use the plotly express bar [chart examples](https://plotly.com/python/bar-charts/#bar-chart-with-sorted-or-ordered-categories) and the [.bar() API reference](https://plotly.com/python-api-reference/generated/plotly.express.bar.html#plotly.express.bar) to create this bar chart: 

<img src=https://imgur.com/LE0XCxA.png>

You'll want to use the `df_free_vs_paid` DataFrame that you created above that has the total number of free and paid apps per category. 

See if you can figure out how to get the look above by changing the `categoryorder` to 'total descending' as outlined in the documentation here [here](https://plotly.com/python/categorical-axes/#automatically-sorting-categories-by-name-or-total-value). 

In [275]:
bar = px.bar(df_free_paid, x='Category', y='App', color='Type', title='Free vs Paid Apps by Category', barmode='group')
bar.update_layout(xaxis_title='Category', yaxis_title='Number of Apps', xaxis={'categoryorder': 'total descending'}, yaxis={'type': 'log'})
bar.show()

# Plotly Box Plots: Lost Downloads for Paid Apps

**Challenge**: Create a box plot that shows the number of Installs for free versus paid apps. How does the median number of installations compare? Is the difference large or small?

Use the [Box Plots Guide](https://plotly.com/python/box-plots/) and the [.box API reference](https://plotly.com/python-api-reference/generated/plotly.express.box.html) to create the following chart. 

<img src=https://imgur.com/uVsECT3.png>


In [276]:
box = px.box(df_sub250, y='Installs', x='Type', color='Type', notched=True, points='all', title='How Many Downloads are Paid Apps Giving Up?')
box.update_layout(yaxis={'type': 'log'})
box.show()

# Plotly Box Plots: Revenue by App Category

**Challenge**: See if you can generate the chart below: 

<img src=https://imgur.com/v4CiNqX.png>

Looking at the hover text, how much does the median app earn in the Tools category? If developing an Android app costs $30,000 or thereabouts, does the average photography app recoup its development costs?

Hint: I've used 'min ascending' to sort the categories. 

In [277]:
df_paid = df_sub250[df_sub250['Type']=='Paid']
box = px.box(df_paid, x='Category', y='Revenue_Estimate', title='How Much Can Paid Apps Earn?')
box.update_layout(xaxis_title='Category', yaxis_title='Paid App Ballpark Revenue', xaxis={'categoryorder': 'min ascending'}, yaxis={'type': 'log'})
box.show()

# How Much Can You Charge? Examine Paid App Pricing Strategies by Category

**Challenge**: What is the median price price for a paid app? Then compare pricing by category by creating another box plot. But this time examine the prices (instead of the revenue estimates) of the paid apps. I recommend using `{categoryorder':'max descending'}` to sort the categories.

In [278]:
box = px.box(df_paid, x='Category', y='Price', title='Price per Category')
box.update_layout(xaxis_title='Category', yaxis_title='Paid App Price', xaxis={'categoryorder': 'max descending'}, yaxis={'type': 'log'})
box.show()